# Логичтическая регрессия, метод опорных векторов, one-hot кодирование

### О задании

В этом задании вы:
- настроите метод опорных векторов
- изучите методы работы с категориальными переменными

In [4]:
%matplotlib inline
import pandas as pd
from joblib import Parallel, delayed
import warnings, time
warnings.simplefilter(action='ignore')
from sklearn.base import BaseEstimator
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import copy
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder

__Задание 1.__ Обучение логистической регрессии на реальных данных и оценка качества классификации.

**(5 баллов)**


Загрузим данные с конкурса [Kaggle Porto Seguro’s Safe Driver Prediction](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction) (вам нужна только обучающая выборка). Задача состоит в определении водителей, которые в ближайший год воспользуются своей автомобильной страховкой (бинарная классификация). Но для нас важна будет не сама задача, а только её данные. При этом под нужды задания мы немного модифицируем датасет.

In [5]:
data = pd.read_csv('train.csv', index_col=0)
target = data.target.values
data = data.drop('target', axis=1)

Пересемплируем выборку так, чтобы положительных и отрицательных объектов в выборке было одинаковое число. Разделим на обучающую и тестовую выборки.

In [6]:
np.random.seed(910)
mask_plus = np.random.choice(np.where(target == 1)[0], 100000, replace=True)
mask_zero = np.random.choice(np.where(target == 0)[0], 100000, replace=True)
mask = np.concatenate([mask_plus, mask_zero])
mask = np.sort(mask)

data = data.iloc[mask]
target = target[mask]

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.5, random_state=73)

Не забудьте отнормировать признаки (можно воспользоваться StandardScaler или сделать это вручную). Пока не будем обращать внимание на то, что некоторые признаки категориальные (этим мы займёмся позже).

In [7]:
def normalize(data):
    new_data = copy.deepcopy(data)
    for c in data.columns:
        new_data[c] = (new_data[c] - new_data[c].min()) / (new_data[c].max() - new_data[c].min())
    return new_data

Обучите логистическую регрессию с удобными для вас параметрами, примените регуляризацию. Сделайте предсказание на тестовой части выборки. Посчитайте accuracy, precision, recall и F меру

In [8]:
def get(X_train, X_test, y_train, y_test):
    st = time.time()
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print(f"Accuracy:", accuracy)
    print(f"Precision:", precision)
    print(f"Recall:", recall)
    print(f"F1 Score:", f1)
    en = time.time()
    print(f"Time taken: {en - st} s")


get(*train_test_split(normalize(data), target, test_size=0.5, random_state=73))


Accuracy: 0.58863
Precision: 0.5971208920443408
Recall: 0.5477465238932395
F1 Score: 0.5713690308732664
Time taken: 0.7139253616333008 s


__Выводы__ в свободной форме:

    Accuracy: 0.59 - доля правильных предсказаний модели, составляет примерно 59%. Это означает, что модель правильно классифицировала около 59% объектов в тестовой выборке.

    Precision: 0.60 - доля правильно предсказанных положительных классов среди всех предсказанных положительных классов, составляет примерно 60%. Это указывает на то, что из объектов, которые модель предсказала как положительные, около 60% действительно являются положительными.

    Recall: 0.55 - доля правильно предсказанных положительных классов среди всех истинных положительных классов, составляет примерно 55%. Это означает, что модель обнаруживает около 55% всех действительных положительных случаев.

    F1 Score: 0.57 - гармоническое среднее между точностью и полнотой составляет примерно 57%. Это важная метрика, особенно в случае дисбаланса классов, так как она учитывает как точность, так и полноту предсказаний.

## Часть 2. Работа с категориальными переменными

В этой части мы научимся обрабатывать категориальные переменные, так как закодировать их в виде чисел недостаточно (это задаёт некоторый порядок, которого на категориальных переменных может и не быть). Существует два основных способа обработки категориальных значений:
- One-hot-кодирование
- Счётчики (CTR, mean-target кодирование, ...) — каждый категориальный признак заменяется на среднее значение целевой переменной по всем объектам, имеющим одинаковое значение в этом признаке.

Начнём с one-hot-кодирования. Допустим наш категориальный признак $f_j(x)$ принимает значения из множества $C=\{c_1, \dots, c_m\}$. Заменим его на $m$ бинарных признаков $b_1(x), \dots, b_m(x)$, каждый из которых является индикатором одного из возможных категориальных значений:
$$
b_i(x) = [f_j(x) = c_i]
$$

__Задание 1.__ Закодируйте все категориальные признаки с помощью one-hot-кодирования. Обучите логистическую регрессию и посмотрите, как изменилось качество модели (с тем, что было ранее). Измерьте время, потребовавшееся на обучение модели.

__(3 балла)__

In [9]:
data.describe()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,1.998215,1.366550,4.483870,0.429490,0.502265,0.351100,0.295375,0.175920,0.177605,0.000470,...,5.443865,1.443745,2.873590,7.544455,0.123355,0.630875,0.553405,0.287530,0.345000,0.152800
std,2.017199,0.674421,2.739255,0.496689,1.501934,0.477315,0.456212,0.380753,0.382181,0.021674,...,2.342462,1.201163,1.692875,2.745287,0.328845,0.482569,0.497141,0.452612,0.475369,0.359796
min,0.000000,-1.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,1.000000,3.000000,7.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,3.000000,2.000000,7.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,7.000000,2.000000,4.000000,9.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,7.000000,4.000000,11.000000,1.000000,6.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,18.000000,8.000000,13.000000,22.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
categorical_features = [c for c in data.columns if c.endswith('_cat')]

encoder = OneHotEncoder(sparse_output=False, drop="first")
encoded_features = encoder.fit_transform(data[categorical_features])

data_one_hot = data.drop(categorical_features, axis=1)
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_features))

data_one_hot.reset_index(inplace=True, drop=True)
data_one_hot = pd.concat([data_one_hot, encoded_df], axis=1)

get(*train_test_split(normalize(data_one_hot), target, test_size=0.5, random_state=73))

Accuracy: 0.59375
Precision: 0.6017653279236895
Recall: 0.5570560971711683
F1 Score: 0.57854823483033
Time taken: 3.7121293544769287 s


Как можно было заметить, one-hot-кодирование может сильно увеличивать количество признаков в датасете, что сказывается на памяти, особенно, если некоторый признак имеет большое количество значений. Эту проблему решает другой способ кодирование категориальных признаков — счётчики. Основная идея в том, что нам важны не сами категории, а значения целевой переменной, которые имеют объекты этой категории. Каждый категориальный признак мы заменим средним значением целевой переменной по всем объектам этой же категории:
$$
g_j(x, X) = \frac{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)][y_i = +1]}{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)]}
$$

__Задание 2.__ Закодируйте категориальные переменные с помощью счётчиков (ровно так, как описано выше без каких-либо хитростей). Обучите логистическую регрессию и посмотрите на качество модели на тестовом множестве. Сравните время обучения с предыдущим экспериментов. Заметили ли вы что-то интересное?

__(2 балла)__

In [11]:
X = data.copy()
y = target.copy()

for feature in categorical_features:
    for i in np.unique(X[feature]):
        X[feature][X[feature] == i] = np.mean(y[X[feature] == i])

get(*train_test_split(X, y, test_size=0.5, random_state=73))

Accuracy: 0.59277
Precision: 0.5992302294426606
Recall: 0.5629694741889084
F1 Score: 0.5805341820916123
Time taken: 15.150947570800781 s


__Вывод:__

    При использовании счётчиков F1 score немного лучше, чем в one-hot-encoding, но на обучение уходит немного больше времени (не чувствительно).

Отметим, что такие признаки сами по себе являются классификаторами и, обучаясь на них, мы допускаем "утечку" целевой переменной в признаки. Это ведёт к переобучению, поэтому считать такие признаки необходимо таким образом, чтобы при вычислении для конкретного объекта его целевая метка не использовалась. Это можно делать следующими способами:
- вычислять значение счётчика по всем объектам расположенным выше в датасете (например, если у нас выборка отсортирована по времени)
- вычислять по фолдам, то есть делить выборку на некоторое количество частей и подсчитывать значение признаков по всем фолдам кроме текущего (как делается в кросс-валидации)
- внесение некоторого шума в посчитанные признаки (необходимо соблюсти баланс между избавление от переобучения и полезностью признаков).

__Задание 3.__ Реализуйте корректное вычисление счётчиков двумя из трех вышеперчисленных способов, сравните. Снова обучите логистическую регрессию, оцените качество. Сделайте выводы.

__(3 балла)__

In [12]:
# по объектам расположенным выше в датасете
X = data.copy()
y = target.copy()
for feature in categorical_features:
    ind = 0
    ans = []
    mp = {}
    for j in X[feature].tolist():
        mp[j] = mp.get(j, [0, 0])
        mp[j][0] += y[ind]
        mp[j][1] += 1
        ans.append(mp[j][0] / mp[j][1])
        ind += 1
    X[feature] = pd.Series(ans, index=X[feature].index)
%time get(*train_test_split(normalize(X), y, test_size=0.5, random_state=73))

Accuracy: 0.596
Precision: 0.6026622437696691
Recall: 0.5662058494486175
F1 Score: 0.5838655185200445
Time taken: 1.5287456512451172 s
CPU times: total: 1.11 s
Wall time: 1.9 s


In [13]:
# по фолдам
X = data.copy()
y = target.copy()
B = int(len(X) ** 0.5)
for feature in categorical_features:
    lst = X[feature].to_list()
    cnts = [{} for j in range(len(X) // B + 2)]
    sums = [{} for j in range(len(X) // B + 2)]
    for i in range(len(X)):
        if len(cnts[i // B]) == 0:
            cnts[i // B + 1] = cnts[i // B].copy()
            sums[i // B + 1] = sums[i // B].copy()
        cnts[i // B + 1][lst[i]] = cnts[i // B + 1].get(lst[i], 0) + 1
        sums[i // B + 1][lst[i]] = sums[i // B + 1].get(lst[i], 0) + y[i]
    ind = 0
    ans = []
    for j in X[feature].tolist():
        sum = sums[ind // B].get(j, 0) + sums[-1].get(j, 0) - sums[ind // B + 1].get(j, 0)
        cnt = cnts[ind // B].get(j, 0) + cnts[-1].get(j, 0) - cnts[ind // B + 1].get(j, 0)
        ans.append(0 if cnt == 0 else sum / cnt)
        ind += 1
    X[feature] = pd.Series(ans, index=X[feature].index)
%time get(*train_test_split(normalize(X), y, test_size=0.5, random_state=73))

Accuracy: 0.60577
Precision: 0.6124352331606218
Recall: 0.5785320441105961
F1 Score: 0.5950010786821586
Time taken: 1.3502376079559326 s
CPU times: total: 1.16 s
Wall time: 1.69 s


__Вывод:__

    Становится намного лучше. По фолдам работает на 1% лучше (по F1 score).

## Часть 2. Метод опорных векторов и калибровка вероятностней

__Задание 1.__ Обучение и применение метода опорных векторов.

__(1 балл)__

Обучите метод опорных векторов (воспользуйтесь готовой реализацией LinearSVC из sklearn). Используйте уже загруженные и обработанные в предыдущей части данные.

In [14]:
from sklearn.svm import LinearSVC

X_train, X_test, y_train, y_test = train_test_split(normalize(X), y, test_size=0.5, random_state=73)

svc = LinearSVC(random_state=73)
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)

На той же тестовой части посчитайте все те же метрики. Что вы можете сказать о полученных результатах?

In [15]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f"Accuracy:", accuracy)
print(f"Precision:", precision)
print(f"Recall:", recall)
print(f"F1 Score:", f1)

Accuracy: 0.60505
Precision: 0.6122388250047824
Recall: 0.5754355122263065
F1 Score: 0.5932669433488151


__Вывод:__

    F1 score у LinearSVC и предыдущей модели (были использованы одинаковые данные) примерно одинаковы. Заметного изменения не видно.

***Report***

    Узнала о новых способах подготовки данных к обучению модели (one-hot-encoding, счётчики). Пообучала модель классификации.
    Лучше всего работали счетчики с заменой на среднее по фолдам. 